In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
sys.setrecursionlimit(5000)

In [3]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
    get_unique_nutzung
)

from compute_ET_green import compute_et_green, calculate_band_std_dev
from compute_ET_blue import (
    compute_et_blue,
    compute_volumetric_et_blue,
    postprocess_et_blue,
)
from add_walter_irrigation_estimates import filter_and_estimate_irrigation

from utils import ee_utils, date_utils
from utils.ee_utils import back_to_float, back_to_int, export_image_to_asset

from typing import List, Tuple

import ee
import geemap

In [ ]:
ee.Initialize(project="thurgau-irrigation")

## Define the region of interest


In [ ]:
# For the canton of Thurgau
cantonal_borders_asset = (
    "projects/thurgau-irrigation/assets/Zuerich/Zuerich_bound"
)

aoi_feature_collection = ee.FeatureCollection(cantonal_borders_asset)
aoi_geometry = aoi_feature_collection.geometry()
aoi_geometry = aoi_geometry.simplify(500)
aoi_buffered = aoi_geometry.buffer(100)

## Importing all assets: land use maps, vegetation periods, WaPOR ET data.


## Sanity check, uncomment to verify things
### Filtering out potentially irrigated crops and refeence rainfed fields


In [ ]:
cantonal_borders = ee.FeatureCollection(
    "projects/thurgau-irrigation/assets/Zuerich/Zuerich_bound"
)

nutzung_collection = ee.FeatureCollection(
    "projects/thurgau-irrigation/assets/ZH_Nutzungsflaechen_2/ZH_Nutzungsflaechen_2022"
)

# Rename the NUTZUNG field to nutzung
nutzung_collection = nutzung_collection.map(
    lambda feature: feature.set("nutzung", feature.get("NUTZUNG"))
)

## Exporting ET Blue maps


In [ ]:
et_green_collection = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Thurgau/ET_green_jurisdiction_refactored_2018-2023"
).map(lambda image: back_to_float(image, 100))

et_green_collection_list = et_green_collection.toList(et_green_collection.size())

# Sanity check
# print(et_green_collection_list.size().getInfo())
# date_utils.print_collection_dates(et_green_collection)

In [ ]:
# TIME_STEPS = 12

# tasks = []

# total_tasks = 0

# previous_ET_image = None

# for year in years_to_process:
#     j = year - 2018

#     year_tasks = 0

#     for i in range(TIME_STEPS):
#         et_image_year = ee.Image(WaPOR_et_collections_list.get(j * TIME_STEPS + i))
#         et_green_year = ee.Image(et_green_collection_list.get(j * TIME_STEPS + i))

#         et_blue = compute_et_blue(et_image_year, et_green_year)

#         # Multiply by 100 and cast to int to save space
#         et_blue = back_to_int(et_blue, 100)

#         if TIME_STEPS == 36:
#             dekadal = i % 3 + 1
#             month = i // 3 + 1
#             time_step_name = f"{month:02d}_D{dekadal}"
#             time_step_type = "dekadal"

#         elif TIME_STEPS == 12:
#             month = i + 1
#             time_step_name = f"{month:02d}"
#             time_step_type = "monthly"

#         else:
#             raise ValueError("Invalid number of time steps. Should be 12 or 36.")

#         task_name = f"thurgau_ET_raw_refactored_{year}-{time_step_name}"

#         asset_id = f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw_wapor_{time_step_type}_2018-2023/{task_name}"

#         task = export_image_to_asset(et_blue, asset_id, task_name, year, aoi_buffered)
#         tasks.append(task)
#         year_tasks += 1
#         total_tasks += 1

#     print(f"Year {year} processing complete. Started {year_tasks} tasks.")

# print(f"All processing complete. Total tasks started: {total_tasks}")

## Postprocess ET_blue

In [ ]:
# TIME_STEPS = 36
# tasks = []
# total_tasks = 0

# et_blue_raw = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw_refactored_2018-2023"
# ).map(lambda image: back_to_float(image, 100))

# et_blue_raw_list = et_blue_raw.toList(et_blue_raw.size())

# # Initialize as None at the start
# et_blue_previous = None

# for year in range(2022, 2024):
#     j = year - 2018
#     year_tasks = 0

#     for i in range(TIME_STEPS):
#         current_index = j * TIME_STEPS + i
#         et_green_year = ee.Image(et_green_collection_list.get(current_index))
#         et_blue_present = ee.Image(et_blue_raw_list.get(current_index))

#         # For the very first image of the series
#         if et_blue_previous is None:
#             et_blue_previous = et_blue_present

#         threshold = calculate_band_std_dev(et_green_year, "ET_green")

#         # Post process using the previous processed image
#         et_blue = postprocess_et_blue(et_blue_present, et_blue_previous, threshold)

#         # Compute volumetric
#         et_blue_m3 = compute_volumetric_et_blue(et_blue)

#         # Add volumetric band
#         et_blue = et_blue.addBands(et_blue_m3)

#         # Store the current processed ET blue for next iteration
#         et_blue_previous = et_blue.select("ET_blue")

#         # Convert to int for storage
#         et_blue = back_to_int(et_blue, 100)

#         # Determine time step format
#         if TIME_STEPS == 36:
#             dekadal = i % 3 + 1
#             month = i // 3 + 1
#             time_step_name = f"{month:02d}_D{dekadal}"
#             time_step_type = "dekadal"
#         elif TIME_STEPS == 12:
#             month = i + 1
#             time_step_name = f"{month:02d}"
#             time_step_type = "monthly"
#         else:
#             raise ValueError("Invalid number of time steps. Should be 12 or 36.")

#         # Create task name and asset ID
#         task_name = (
#             f"ET_blue_postprocessed_refactored_{time_step_type}_{year}-{time_step_name}"
#         )
#         asset_id = f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed_wapor_{time_step_type}_refactored_2018-2023/{task_name}"

#         # Export
#         task = export_image_to_asset(et_blue, asset_id, task_name, year, aoi_buffered)
#         tasks.append(task)
#         year_tasks += 1
#         total_tasks += 1

#     print(f"Year {year} processing complete. Started {year_tasks} tasks.")

# print(f"All processing complete. Total tasks started: {total_tasks}")

In [ ]:
# et_green = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/ET_green_jurisdiction_refactored_2018-2023").map(lambda image: back_to_float(image, 100))

# et_green_list = et_green.toList(et_green.size())

# for i in range(et_green.size().getInfo()):
#     image = ee.Image(et_green_list.get(i))
#     print(calculate_band_std_dev(image, "ET_green").getInfo())


## Exploring Nutzungsflaechen


In [ ]:
# nutzung_collection = ee.FeatureCollection(
#     "projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_2018_area"
# )

In [ ]:
# def get_unique_nutzung_values(feature_collection: ee.FeatureCollection) -> list:
#     """
#     Get all unique values of the 'nutzung' property from a feature collection.

#     Args:
#         feature_collection (ee.FeatureCollection): The input feature collection.

#     Returns:
#         list: A list of unique 'nutzung' values.
#     """
#     # Get distinct values of 'nutzung'
#     distinct_values = feature_collection.distinct('nutzung')

#     # Aggregate the distinct values into a list
#     unique_values = distinct_values.aggregate_array('nutzung')

#     # Get the result as a Python list
#     return unique_values.getInfo()

# unique_nutzung_values = get_unique_nutzung_values(nutzung_collection)

# print("Unique 'nutzung' values:")
# for value in unique_nutzung_values:
#     print(value)

In [ ]:
# def export_feature_collection(collection: ee.FeatureCollection, task_name: str, asset_id: str):
#     """
#     Export the feature collection to an Earth Engine asset.

#     Args:
#         collection: The feature collection to export.
#         year: The year of the feature collection.
#         task_name: The name of the export task.
#         asset_id: The asset ID to export to.
#     """
#     task = ee.batch.Export.table.toAsset(
#         collection=collection,
#         description=task_name,
#         assetId=asset_id,
#     )
#     task.start()

# for year in years_to_process:
#     nutzung_collection_year = ee.FeatureCollection(
#         f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{year}_area"
#     )

#     # Apply the filter and estimation function
#     validation_nutzung_collection_year = filter_and_estimate_irrigation(nutzung_collection_year)

#     task_name = f"irrigation_estimates_{year}"
#     asset_id = f"projects/thurgau-irrigation/assets/Thurgau/Walter_Koch_irrigation_estimates_2018-2022/{task_name}"

#     export_feature_collection(validation_nutzung_collection_year, task_name, asset_id)

# print("All processing complete.")

In [ ]:
# image_collection = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw_wapor_monthly_2018-2023").map(lambda image: back_to_float(image, 100))

# image_list = image_collection.toList(image_collection.size())

# image = ee.Image(image_list.get(56))

# # image = ee.Image(WaPOR_et_collections_list.get(15)).subtract(ee.Image(et_green_collection_list.get(15)))

# Map = geemap.Map()

# Map.centerObject(aoi_geometry, 12)

# vis_params = {
#     "bands": ["ET_blue"],
#     "min": -10,
#     "max": 20,
#     "palette": ["white", "blue", "green", "yellow", "red"],
# }

# Map.addLayer(image, vis_params, "ET_blue_raw")

# Map

In [ ]:
# et_blue_raw = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw_2018-2022"
# ).map(lambda image: back_to_float(image, 100))

# et_blue_raw_list = et_blue_raw.toList(et_blue_raw.size())

In [ ]:
# et_blue_present = ee.Image(et_blue_raw_list.get(20))
# et_blue_previous = ee.Image(et_blue_raw_list.get(19))

# et_green_year = ee.Image(et_green_collection_list.get(20))

# threshold = calculate_band_std_dev(et_green_year, "ET_green")

# et_blue = postprocess_et_blue(et_blue_present, et_blue_previous, threshold)

# et_blue_m3 = compute_volumetric_et_blue(et_blue)

In [ ]:
# image_postprocessed = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed_2018-2022")

# image_postprocessed.first().date().format("YYYY-MM-dd").getInfo()